In [ ]:
# Preprocessing Cell
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Load dataset
file_path = r"C:\Users\user\Desktop\CW4\online+retail\Online Retail.xlsx"
df = pd.read_excel(file_path, sheet_name='Online Retail')

# Cleaning
print(f"Original Data Shape: {df.shape}")
df_cleaned = df.dropna(subset=['CustomerID', 'Description'])
df_cleaned = df_cleaned[df_cleaned['Quantity'] > 0]
print(f"Cleaned Data Shape: {df_cleaned.shape}")

# Encoding
df_sorted = df_cleaned.sort_values(by=['CustomerID', 'InvoiceDate'])
all_items = df_sorted['Description'].unique()
item_encoder = LabelEncoder()
item_encoder.fit(all_items)

# Encode ItemID
df_sorted['ItemID'] = item_encoder.transform(df_sorted['Description'])
print(f"ItemID range after encoding: {df_sorted['ItemID'].min()} to {df_sorted['ItemID'].max()}")

# Grouping by Customer
sequential_data = df_sorted.groupby('CustomerID')['ItemID'].apply(list).reset_index(name='ItemSequence')

# Minimum sequence length
min_sequence_length = 3
sequential_data = sequential_data[sequential_data['ItemSequence'].apply(len) >= min_sequence_length]
print(f"Data Shape after filtering short sequences: {sequential_data.shape}")

# Pad and create sequences
item_sequences = sequential_data['ItemSequence'].tolist()
sequence_length = 30
padded_sequences = pad_sequences(item_sequences, maxlen=sequence_length, padding='pre')

def create_sequences(sequences, seq_length=30):
    X, y = [], []
    for seq in sequences:
        for i in range(max(1, len(seq) - seq_length + 1)):
            X.append(seq[i:i + seq_length])
            y.append(seq[min(i + seq_length, len(seq) - 1)])  # Adjust for boundaries
    return np.array(X), np.array(y)

X, y = create_sequences(padded_sequences, seq_length=sequence_length)

# Dataset splitting
split_ratio = 0.8
split_index = int(len(X) * split_ratio)
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

# Dynamically calculate the unique item count
unique_items = np.unique(np.concatenate([X.flatten(), y]))
num_items = len(unique_items)

# Fit LabelEncoder on all items in both X and y
unique_items = np.unique(np.concatenate([X.flatten(), y]))
item_encoder = LabelEncoder()
item_encoder.fit(unique_items)

# Re-encode X and y to ensure compatibility
X_flat = X.flatten()
X_encoded = item_encoder.transform(X_flat).reshape(X.shape)
y_encoded = item_encoder.transform(y)

# Update the dataset
X_train, X_test = X_encoded[:split_index], X_encoded[split_index:]
y_train, y_test = y_encoded[:split_index], y_encoded[split_index:]

# Recalculate vocabulary size
num_items = len(item_encoder.classes_)

# Debugging information
print(f"Recalculated Vocabulary Size: {num_items}")
print(f"X_train shape: {X_train.shape}, X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}, y_test shape: {y_test.shape}")
print(f"X_train range: {X_train.min()} to {X_train.max()}")
print(f"y_train range: {y_train.min()} to {y_train.max()}")

# Validate data integrity
assert X_train.max() < num_items, "X_train contains indices exceeding vocabulary size!"
assert y_train.max() < num_items, "y_train contains indices exceeding vocabulary size!"

Original Data Shape: (541909, 8)
Cleaned Data Shape: (397924, 8)
ItemID range after encoding: 0 to 3876
Data Shape after filtering short sequences: (4214, 2)
Recalculated Vocabulary Size: 3463
X_train shape: (3371, 30), X_test shape: (843, 30)
y_train shape: (3371,), y_test shape: (843,)
X_train range: 0 to 3462
y_train range: 1 to 3460


In [ ]:
# CPU Optimization Settings
import os
os.environ['OMP_NUM_THREADS'] = '16'
os.environ['TF_NUM_INTRA_OP_PARALLELISM_THREADS'] = '16'
os.environ['TF_NUM_INTER_OP_PARALLELISM_THREADS'] = '8'

import time
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseTopKCategoricalAccuracy
from tqdm import tqdm
import pandas as pd

# Define the SASRec model
class SASRec(tf.keras.Model):
    def __init__(self, input_dim, sequence_length, embedding_dim, num_heads, num_blocks):
        super(SASRec, self).__init__()
        self.embedding_layer = tf.keras.layers.Embedding(input_dim, embedding_dim, mask_zero=True)
        self.encoder = [
            tf.keras.layers.MultiHeadAttention(
                num_heads=num_heads,
                key_dim=embedding_dim,
                dropout=0.1
            )
            for _ in range(num_blocks)
        ]
        self.dense = tf.keras.layers.Dense(input_dim, activation="softmax")
        self.sequence_length = sequence_length

    def call(self, inputs):
        x = self.embedding_layer(inputs)
        for block in self.encoder:
            x = block(x, x)
        # Focus on the last timestep for prediction
        x = x[:, -1, :]  # Shape becomes (batch_size, embedding_dim)
        return self.dense(x)  # Shape becomes (batch_size, input_dim)

# Build the SASRec model dynamically
model = SASRec(
    input_dim=num_items,  # Updated vocabulary size
    sequence_length=30,
    embedding_dim=50,
    num_heads=2,
    num_blocks=2
)

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=SparseCategoricalCrossentropy(),
    metrics=[SparseTopKCategoricalAccuracy(k=10)]
)

# Training Loop
epochs = 10
batch_size = 32
metrics_history = []

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    start_time = time.time()
    epoch_loss = 0
    steps_done = 0
    total_steps = len(X_train) // batch_size + (len(X_train) % batch_size != 0)

    with tqdm(total=total_steps, desc=f"Epoch {epoch + 1}", dynamic_ncols=True) as pbar:
        for step in range(0, len(X_train), batch_size):
            batch_x = X_train[step:step + batch_size]
            batch_y = y_train[step:step + batch_size]

            # Convert to tensors
            batch_x = tf.convert_to_tensor(batch_x, dtype=tf.int32)
            batch_y = tf.convert_to_tensor(batch_y, dtype=tf.int32)

            try:
                # Train on batch
                metrics = model.train_on_batch(batch_x, batch_y)
                epoch_loss += metrics[0]

                # Update progress dynamically
                pbar.set_postfix(loss=metrics[0])
                pbar.update(1)

            except Exception as e:
                print(f"\nError during training at step {steps_done}: {e}")
                break  # Stop training if an error occurs

    # End of epoch
    elapsed_time = time.time() - start_time
    epoch_avg_loss = epoch_loss / total_steps
    print(f"\nEpoch {epoch + 1} Loss: {epoch_avg_loss:.4f} - Time: {elapsed_time:.2f}s")
    metrics_history.append({"Epoch": epoch + 1, "Loss": epoch_avg_loss, "Time (s)": elapsed_time})

# Model Evaluation
test_loss, test_accuracy = model.evaluate(X_test, y_test, batch_size=batch_size)
print(f"\nTest Loss: {test_loss:.4f}, Test Top-10 Accuracy: {test_accuracy:.4f}")

# Print All Metrics
print("\n=== Final Metrics ===")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Top-10 Accuracy: {test_accuracy:.4f}")
print(f"Training History:")
for metric in metrics_history:
    print(f"Epoch {metric['Epoch']} - Loss: {metric['Loss']:.4f} - Time: {metric['Time (s)']:.2f}s")

# Save Results and Model
results = {
    "Test Loss": test_loss,
    "Test Top-10 Accuracy": test_accuracy
}
results_df = pd.DataFrame([results])
results_df.to_csv(r"C:\Users\user\Desktop\CW4\online+retail\sasrec_results.csv", index=False)
print(f"\nResults saved to sasrec_results.csv")


# Collect all results
all_experiment_results = []

# After each experiment, append results
experiment_details = {
    "Experiment": "Experiment 1",
    "Embedding Dim": 50,
    "Num Heads": 2,
    "Num Blocks": 2,
    "Batch Size": 32,
    "Learning Rate": 0.001,
    "Dropout Rate": 0.1,
    "Epochs": epoch + 1,  # Actual number of epochs run (consider early stopping)
    "Test Loss": test_loss,
    "Test Top-10 Accuracy": test_accuracy
}
all_experiment_results.append(experiment_details)

# At the end of all experiments
results_df = pd.DataFrame(all_experiment_results)
results_df.to_csv(r"C:\Users\user\Desktop\CW4\online+retail\SASRec_results1.csv", index=False)
print("\n=== All Experiment Results ===")
print(results_df)


Epoch 1/10


Epoch 1:   0%|                                                                                 | 0/106 [00:00<?, ?it/s]C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:932: UserWarning: Layer 'query' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:932: UserWarning: Layer 'key' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:932: UserWarning: Layer 'value' (of type EinsumDense) was passed an input with a 

Epoch 1:   5%|██▋                                                       | 5/106 [00:03<00:53,  1.89it/s, loss=8.142521]

Epoch 1: 100%|███████████████████████████████████████████████████████| 106/106 [00:44<00:00,  2.36it/s, loss=7.7747364]



Epoch 1 Loss: 7.9652 - Time: 44.92s
Epoch 2/10


...


Epoch 8 Loss: 6.6444 - Time: 57.26s
Epoch 9/10


Epoch 9: 100%|███████████████████████████████████████████████████████| 106/106 [01:00<00:00,  1.76it/s, loss=6.6106253]



Epoch 9 Loss: 6.6081 - Time: 60.37s
Epoch 10/10


Epoch 10: 100%|██████████████████████████████████████████████████████| 106/106 [01:01<00:00,  1.71it/s, loss=6.5780797]



Epoch 10 Loss: 6.5763 - Time: 61.97s
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 8.6793 - sparse_top_k_categorical_accuracy: 0.0487

Test Loss: 8.6643, Test Top-10 Accuracy: 0.0676

=== Final Metrics ===
Test Loss: 8.6643
Test Top-10 Accuracy: 0.0676
Training History:
Epoch 1 - Loss: 7.9652 - Time: 44.92s
Epoch 2 - Loss: 7.3664 - Time: 45.43s
Epoch 3 - Loss: 7.0499 - Time: 49.40s
Epoch 4 - Loss: 6.8929 - Time: 50.95s
Epoch 5 - Loss: 6.8009 - Time: 51.53s
Epoch 6 - Loss: 6.7375 - Time: 53.70s
Epoch 7 - Loss: 6.6868 - Time: 55.56s
Epoch 8 - Loss: 6.6444 - Time: 57.26s
Epoch 9 - Loss: 6.6081 - Time: 60.37s
Epoch 10 - Loss: 6.5763 - Time: 61.97s

Results saved to sasrec_results.csv

=== All Experiment Results ===
     Experiment  Embedding Dim  Num Heads  Num Blocks  Batch Size  \
0  Experiment 1             50          2           2          32   

   Learning Rate  Dropout Rate  Epochs  Test Loss  Test Top-10 Accuracy  
0          0.001           0.1      10    8.66431              

In [ ]:
# CPU Optimization Settings
import os
os.environ['OMP_NUM_THREADS'] = '16'
os.environ['TF_NUM_INTRA_OP_PARALLELISM_THREADS'] = '16'
os.environ['TF_NUM_INTER_OP_PARALLELISM_THREADS'] = '8'

import time
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseTopKCategoricalAccuracy
from tqdm import tqdm
import pandas as pd
import numpy as np

# Define the SASRec model
class SASRec(tf.keras.Model):
    def __init__(self, input_dim, sequence_length, embedding_dim, num_heads, num_blocks):
        super(SASRec, self).__init__()
        self.embedding_layer = tf.keras.layers.Embedding(input_dim, embedding_dim, mask_zero=True)
        self.encoder = [
            tf.keras.layers.MultiHeadAttention(
                num_heads=num_heads,
                key_dim=embedding_dim,
                dropout=0.1
            )
            for _ in range(num_blocks)
        ]
        self.dense = tf.keras.layers.Dense(input_dim, activation="softmax")
        self.sequence_length = sequence_length

    def call(self, inputs):
        x = self.embedding_layer(inputs)
        for block in self.encoder:
            x = block(x, x)
        # Focus on the last timestep for prediction
        x = x[:, -1, :]  # Shape becomes (batch_size, embedding_dim)
        return self.dense(x)  # Shape becomes (batch_size, input_dim)

# Custom metrics: Precision@5, Recall@5, Hit Rate, and MRR
def calculate_metrics(y_true, y_pred, k=5):
    """
    Calculate Precision@K, Recall@K, Hit Rate, and MRR for a batch.
    """
    precision_at_k, recall_at_k, hit_rate, mrr = 0, 0, 0, 0
    batch_size = y_true.shape[0]

    for i in range(batch_size):
        true_label = y_true[i]
        top_k_indices = np.argsort(y_pred[i])[-k:][::-1]

        if true_label in top_k_indices:
            rank = np.where(top_k_indices == true_label)[0][0] + 1
            precision_at_k += 1 / k
            recall_at_k += 1
            hit_rate += 1
            mrr += 1 / rank

    return (
        precision_at_k / batch_size,
        recall_at_k / batch_size,
        hit_rate / batch_size,
        mrr / batch_size,
    )

# Build the SASRec model dynamically
model = SASRec(
    input_dim=num_items,  # Updated vocabulary size
    sequence_length=30,
    embedding_dim=32,
    num_heads=2,
    num_blocks=2
)

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=SparseCategoricalCrossentropy(),
    metrics=[SparseTopKCategoricalAccuracy(k=10)]
)

# Early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=3,
    restore_best_weights=True
)

# Training Loop
epochs = 20
batch_size = 32
metrics_history = []

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    start_time = time.time()
    epoch_loss = 0
    steps_done = 0
    total_steps = len(X_train) // batch_size + (len(X_train) % batch_size != 0)

    with tqdm(total=total_steps, desc=f"Epoch {epoch + 1}", dynamic_ncols=True) as pbar:
        for step in range(0, len(X_train), batch_size):
            batch_x = X_train[step:step + batch_size]
            batch_y = y_train[step:step + batch_size]

            # Convert to tensors
            batch_x = tf.convert_to_tensor(batch_x, dtype=tf.int32)
            batch_y = tf.convert_to_tensor(batch_y, dtype=tf.int32)

            try:
                # Train on batch
                metrics = model.train_on_batch(batch_x, batch_y)
                epoch_loss += metrics[0]

                # Update progress dynamically
                pbar.set_postfix(loss=metrics[0])
                pbar.update(1)

            except Exception as e:
                print(f"\nError during training at step {steps_done}: {e}")
                break  # Stop training if an error occurs

    # End of epoch
    elapsed_time = time.time() - start_time
    epoch_avg_loss = epoch_loss / total_steps
    print(f"\nEpoch {epoch + 1} Loss: {epoch_avg_loss:.4f} - Time: {elapsed_time:.2f}s")
    metrics_history.append({"Epoch": epoch + 1, "Loss": epoch_avg_loss, "Time (s)": elapsed_time})

# Model Evaluation
test_loss, test_top_10_accuracy = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)
y_pred = model.predict(X_test, batch_size=batch_size)

# Calculate additional metrics
precision_at_5, recall_at_5, hit_rate, mrr = calculate_metrics(y_test, y_pred, k=5)

# Print All Metrics
print("\n=== Final Metrics ===")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Top-10 Accuracy: {test_top_10_accuracy:.4f}")
print(f"Precision@5: {precision_at_5:.4f}")
print(f"Recall@5: {recall_at_5:.4f}")
print(f"Hit Rate: {hit_rate:.4f}")
print(f"MRR: {mrr:.4f}")

# Save Results and Model
results = {
    "Experiment": "Experiment Template",
    "Embedding Dim": 50,
    "Num Heads": 4,
    "Num Blocks": 2,
    "Batch Size": 32,
    "Learning Rate": 0.001,
    "Dropout Rate": 0.1,
    "Epochs": len(metrics_history),  # Actual number of epochs run
    "Test Loss": test_loss,
    "Test Top-10 Accuracy": test_top_10_accuracy,
    "Precision@5": precision_at_5,
    "Recall@5": recall_at_5,
    "Hit Rate": hit_rate,
    "MRR": mrr
}

results_df = pd.DataFrame([results])
results_df.to_csv(r"C:\Users\user\Desktop\CW4\online+retail\sasrec_results.csv", index=False)
print(f"\nResults saved to sasrec_results.csv")


Epoch 1/20


Epoch 1:   0%|                                                                                 | 0/106 [00:00<?, ?it/s]C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:932: UserWarning: Layer 'query' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:932: UserWarning: Layer 'key' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:932: UserWarning: Layer 'value' (of type EinsumDense) was passed an input with a 

Epoch 1:   5%|██▋                                                       | 5/106 [00:02<00:33,  3.00it/s, loss=8.145113]

Epoch 1: 100%|█████████████████████████████████████████████████████████| 106/106 [00:27<00:00,  3.90it/s, loss=7.80733]



Epoch 1 Loss: 8.0078 - Time: 27.15s
Epoch 2/20


...


Epoch 18 Loss: 6.2632 - Time: 48.50s
Epoch 19/20


Epoch 19: 100%|███████████████████████████████████████████████████████| 106/106 [00:50<00:00,  2.09it/s, loss=6.230219]



Epoch 19 Loss: 6.2343 - Time: 50.76s
Epoch 20/20


Epoch 20: 100%|██████████████████████████████████████████████████████| 106/106 [00:52<00:00,  2.03it/s, loss=6.2074084]



Epoch 20 Loss: 6.2096 - Time: 52.12s
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step

=== Final Metrics ===
Test Loss: 11.3258
Test Top-10 Accuracy: 0.0510
Precision@5: 0.0071
Recall@5: 0.0356
Hit Rate: 0.0356
MRR: 0.0283

Results saved to sasrec_results.csv


In [ ]:
# CPU Optimization Settings
import os
os.environ['OMP_NUM_THREADS'] = '16'
os.environ['TF_NUM_INTRA_OP_PARALLELISM_THREADS'] = '16'
os.environ['TF_NUM_INTER_OP_PARALLELISM_THREADS'] = '8'

import time
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseTopKCategoricalAccuracy
from tqdm import tqdm
import pandas as pd
import numpy as np

# Define the SASRec model
class SASRec(tf.keras.Model):
    def __init__(self, input_dim, sequence_length, embedding_dim, num_heads, num_blocks):
        super(SASRec, self).__init__()
        self.embedding_layer = tf.keras.layers.Embedding(input_dim, embedding_dim, mask_zero=True)
        self.encoder = [
            tf.keras.layers.MultiHeadAttention(
                num_heads=num_heads,
                key_dim=embedding_dim,
                dropout=0.1
            )
            for _ in range(num_blocks)
        ]
        self.dense = tf.keras.layers.Dense(input_dim, activation="softmax")
        self.sequence_length = sequence_length

    def call(self, inputs):
        x = self.embedding_layer(inputs)
        for block in self.encoder:
            x = block(x, x)
        # Focus on the last timestep for prediction
        x = x[:, -1, :]  # Shape becomes (batch_size, embedding_dim)
        return self.dense(x)  # Shape becomes (batch_size, input_dim)

# Custom metrics: Precision@5, Recall@5, Hit Rate, and MRR
def calculate_metrics(y_true, y_pred, k=5):
    """
    Calculate Precision@K, Recall@K, Hit Rate, and MRR for a batch.
    """
    precision_at_k, recall_at_k, hit_rate, mrr = 0, 0, 0, 0
    batch_size = y_true.shape[0]

    for i in range(batch_size):
        true_label = y_true[i]
        top_k_indices = np.argsort(y_pred[i])[-k:][::-1]

        if true_label in top_k_indices:
            rank = np.where(top_k_indices == true_label)[0][0] + 1
            precision_at_k += 1 / k
            recall_at_k += 1
            hit_rate += 1
            mrr += 1 / rank

    return (
        precision_at_k / batch_size,
        recall_at_k / batch_size,
        hit_rate / batch_size,
        mrr / batch_size,
    )

# Build the SASRec model dynamically
model = SASRec(
    input_dim=num_items,  # Updated vocabulary size
    sequence_length=30,
    embedding_dim=50,
    num_heads=3,
    num_blocks=2
)

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=SparseCategoricalCrossentropy(),
    metrics=[SparseTopKCategoricalAccuracy(k=10)]
)

# Early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=4,
    restore_best_weights=True
)

# Training Loop
epochs = 40
batch_size = 32
metrics_history = []

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    start_time = time.time()
    epoch_loss = 0
    steps_done = 0
    total_steps = len(X_train) // batch_size + (len(X_train) % batch_size != 0)

    with tqdm(total=total_steps, desc=f"Epoch {epoch + 1}", dynamic_ncols=True) as pbar:
        for step in range(0, len(X_train), batch_size):
            batch_x = X_train[step:step + batch_size]
            batch_y = y_train[step:step + batch_size]

            # Convert to tensors
            batch_x = tf.convert_to_tensor(batch_x, dtype=tf.int32)
            batch_y = tf.convert_to_tensor(batch_y, dtype=tf.int32)

            try:
                # Train on batch
                metrics = model.train_on_batch(batch_x, batch_y)
                epoch_loss += metrics[0]

                # Update progress dynamically
                pbar.set_postfix(loss=metrics[0])
                pbar.update(1)

            except Exception as e:
                print(f"\nError during training at step {steps_done}: {e}")
                break  # Stop training if an error occurs

    # End of epoch
    elapsed_time = time.time() - start_time
    epoch_avg_loss = epoch_loss / total_steps
    print(f"\nEpoch {epoch + 1} Loss: {epoch_avg_loss:.4f} - Time: {elapsed_time:.2f}s")
    metrics_history.append({"Epoch": epoch + 1, "Loss": epoch_avg_loss, "Time (s)": elapsed_time})

# Model Evaluation
test_loss, test_top_10_accuracy = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)
y_pred = model.predict(X_test, batch_size=batch_size)

# Calculate additional metrics
precision_at_5, recall_at_5, hit_rate, mrr = calculate_metrics(y_test, y_pred, k=5)

# Print All Metrics
print("\n=== Final Metrics ===")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Top-10 Accuracy: {test_top_10_accuracy:.4f}")
print(f"Precision@5: {precision_at_5:.4f}")
print(f"Recall@5: {recall_at_5:.4f}")
print(f"Hit Rate: {hit_rate:.4f}")
print(f"MRR: {mrr:.4f}")

# Save Results and Model
results = {
    "Experiment": "Experiment Template",
    "Embedding Dim": 50,
    "Num Heads": 4,
    "Num Blocks": 2,
    "Batch Size": 32,
    "Learning Rate": 0.001,
    "Dropout Rate": 0.1,
    "Epochs": len(metrics_history),  # Actual number of epochs run
    "Test Loss": test_loss,
    "Test Top-10 Accuracy": test_top_10_accuracy,
    "Precision@5": precision_at_5,
    "Recall@5": recall_at_5,
    "Hit Rate": hit_rate,
    "MRR": mrr
}

results_df = pd.DataFrame([results])
results_df.to_csv(r"C:\Users\user\Desktop\CW4\online+retail\sasrec_results13.csv", index=False)
print(f"\nResults saved to sasrec_results13.csv")


Epoch 1/40


Epoch 1:   0%|                                                                                 | 0/106 [00:00<?, ?it/s]C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:932: UserWarning: Layer 'query' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:932: UserWarning: Layer 'key' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:932: UserWarning: Layer 'value' (of type EinsumDense) was passed an input with a 

Epoch 1:   5%|██▋                                                       | 5/106 [00:03<00:53,  1.88it/s, loss=8.142586]

Epoch 1: 100%|███████████████████████████████████████████████████████| 106/106 [00:44<00:00,  2.40it/s, loss=7.7697906]



Epoch 1 Loss: 7.9569 - Time: 44.26s
Epoch 2/40


...


Epoch 38 Loss: 5.7935 - Time: 129.06s
Epoch 39/40


Epoch 39: 100%|███████████████████████████████████████████████████████| 106/106 [02:12<00:00,  1.25s/it, loss=5.746757]



Epoch 39 Loss: 5.7600 - Time: 132.60s
Epoch 40/40


Epoch 40: 100%|██████████████████████████████████████████████████████| 106/106 [02:11<00:00,  1.24s/it, loss=5.7103214]



Epoch 40 Loss: 5.7249 - Time: 131.96s
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step

=== Final Metrics ===
Test Loss: 15.1248
Test Top-10 Accuracy: 0.0593
Precision@5: 0.0095
Recall@5: 0.0474
Hit Rate: 0.0474
MRR: 0.0351

Results saved to sasrec_results13.csv


In [ ]:
# CPU Optimization Settings
import os
os.environ['OMP_NUM_THREADS'] = '16'
os.environ['TF_NUM_INTRA_OP_PARALLELISM_THREADS'] = '16'
os.environ['TF_NUM_INTER_OP_PARALLELISM_THREADS'] = '8'

import time
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseTopKCategoricalAccuracy
from tqdm import tqdm
import pandas as pd

# Define the SASRec model
class SASRec(tf.keras.Model):
    def __init__(self, input_dim, sequence_length, embedding_dim, num_heads, num_blocks):
        super(SASRec, self).__init__()
        self.embedding_layer = tf.keras.layers.Embedding(input_dim, embedding_dim, mask_zero=True)
        self.encoder = [
            tf.keras.layers.MultiHeadAttention(
                num_heads=num_heads,
                key_dim=embedding_dim,
                dropout=0.1
            )
            for _ in range(num_blocks)
        ]
        self.dense = tf.keras.layers.Dense(input_dim, activation="softmax")
        self.sequence_length = sequence_length

    def call(self, inputs):
        x = self.embedding_layer(inputs)
        for block in self.encoder:
            x = block(x, x)
        # Focus on the last timestep for prediction
        x = x[:, -1, :]  # Shape becomes (batch_size, embedding_dim)
        return self.dense(x)  # Shape becomes (batch_size, input_dim)

# Build the SASRec model dynamically
model = SASRec(
    input_dim=num_items,  # Updated vocabulary size
    sequence_length=30,
    embedding_dim=50,
    num_heads=2,
    num_blocks=2
)

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=SparseCategoricalCrossentropy(),
    metrics=[SparseTopKCategoricalAccuracy(k=10)]
)

# Training Loop
epochs = 10
batch_size = 32
metrics_history = []

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    start_time = time.time()
    epoch_loss = 0
    steps_done = 0
    total_steps = len(X_train) // batch_size + (len(X_train) % batch_size != 0)

    with tqdm(total=total_steps, desc=f"Epoch {epoch + 1}", dynamic_ncols=True) as pbar:
        for step in range(0, len(X_train), batch_size):
            batch_x = X_train[step:step + batch_size]
            batch_y = y_train[step:step + batch_size]

            # Convert to tensors
            batch_x = tf.convert_to_tensor(batch_x, dtype=tf.int32)
            batch_y = tf.convert_to_tensor(batch_y, dtype=tf.int32)

            try:
                # Train on batch
                metrics = model.train_on_batch(batch_x, batch_y)
                epoch_loss += metrics[0]

                # Update progress dynamically
                pbar.set_postfix(loss=metrics[0])
                pbar.update(1)

            except Exception as e:
                print(f"\nError during training at step {steps_done}: {e}")
                break  # Stop training if an error occurs

    # End of epoch
    elapsed_time = time.time() - start_time
    epoch_avg_loss = epoch_loss / total_steps
    print(f"\nEpoch {epoch + 1} Loss: {epoch_avg_loss:.4f} - Time: {elapsed_time:.2f}s")
    metrics_history.append({"Epoch": epoch + 1, "Loss": epoch_avg_loss, "Time (s)": elapsed_time})

# Model Evaluation
test_loss, test_accuracy = model.evaluate(X_test, y_test, batch_size=batch_size)
print(f"\nTest Loss: {test_loss:.4f}, Test Top-10 Accuracy: {test_accuracy:.4f}")

# Print All Metrics
print("\n=== Final Metrics ===")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Top-10 Accuracy: {test_accuracy:.4f}")
print(f"Training History:")
for metric in metrics_history:
    print(f"Epoch {metric['Epoch']} - Loss: {metric['Loss']:.4f} - Time: {metric['Time (s)']:.2f}s")

# Save Results and Model
results = {
    "Test Loss": test_loss,
    "Test Top-10 Accuracy": test_accuracy
}
results_df = pd.DataFrame([results])
results_df.to_csv(r"C:\Users\user\Desktop\CW4\online+retail\sasrec_results.csv", index=False)
print(f"\nResults saved to sasrec_results.csv")


# Collect all results
all_experiment_results = []

# After each experiment, append results
experiment_details = {
    "Experiment": "Experiment 1",
    "Embedding Dim": 50,
    "Num Heads": 2,
    "Num Blocks": 2,
    "Batch Size": 32,
    "Learning Rate": 0.001,
    "Dropout Rate": 0.1,
    "Epochs": epoch + 1,  # Actual number of epochs run (consider early stopping)
    "Test Loss": test_loss,
    "Test Top-10 Accuracy": test_accuracy
}
all_experiment_results.append(experiment_details)

# At the end of all experiments
results_df = pd.DataFrame(all_experiment_results)
results_df.to_csv(r"C:\Users\user\Desktop\CW4\online+retail\all_experiment_results.csv", index=False)
print("\n=== All Experiment Results ===")
print(results_df)


Epoch 1/10


Epoch 1:   0%|                                                                                 | 0/106 [00:00<?, ?it/s]C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:932: UserWarning: Layer 'query' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:932: UserWarning: Layer 'key' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:932: UserWarning: Layer 'value' (of type EinsumDense) was passed an input with a 

Epoch 1:   5%|██▋                                                       | 5/106 [00:03<00:53,  1.89it/s, loss=8.142521]

Epoch 1: 100%|███████████████████████████████████████████████████████| 106/106 [00:44<00:00,  2.36it/s, loss=7.7747364]



Epoch 1 Loss: 7.9652 - Time: 44.92s
Epoch 2/10


...


Epoch 8 Loss: 6.6444 - Time: 57.26s
Epoch 9/10


Epoch 9: 100%|███████████████████████████████████████████████████████| 106/106 [01:00<00:00,  1.76it/s, loss=6.6106253]



Epoch 9 Loss: 6.6081 - Time: 60.37s
Epoch 10/10


Epoch 10: 100%|██████████████████████████████████████████████████████| 106/106 [01:01<00:00,  1.71it/s, loss=6.5780797]



Epoch 10 Loss: 6.5763 - Time: 61.97s
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 8.6793 - sparse_top_k_categorical_accuracy: 0.0487

Test Loss: 8.6643, Test Top-10 Accuracy: 0.0676

=== Final Metrics ===
Test Loss: 8.6643
Test Top-10 Accuracy: 0.0676
Training History:
Epoch 1 - Loss: 7.9652 - Time: 44.92s
Epoch 2 - Loss: 7.3664 - Time: 45.43s
Epoch 3 - Loss: 7.0499 - Time: 49.40s
Epoch 4 - Loss: 6.8929 - Time: 50.95s
Epoch 5 - Loss: 6.8009 - Time: 51.53s
Epoch 6 - Loss: 6.7375 - Time: 53.70s
Epoch 7 - Loss: 6.6868 - Time: 55.56s
Epoch 8 - Loss: 6.6444 - Time: 57.26s
Epoch 9 - Loss: 6.6081 - Time: 60.37s
Epoch 10 - Loss: 6.5763 - Time: 61.97s

Results saved to sasrec_results.csv

=== All Experiment Results ===
     Experiment  Embedding Dim  Num Heads  Num Blocks  Batch Size  \
0  Experiment 1             50          2           2          32   

   Learning Rate  Dropout Rate  Epochs  Test Loss  Test Top-10 Accuracy  
0          0.001           0.1      10    8.66431              

In [ ]:
# CPU Optimization Settings
import os
os.environ['OMP_NUM_THREADS'] = '16'
os.environ['TF_NUM_INTRA_OP_PARALLELISM_THREADS'] = '16'
os.environ['TF_NUM_INTER_OP_PARALLELISM_THREADS'] = '8'

import time
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseTopKCategoricalAccuracy
from tqdm import tqdm
import pandas as pd

# Define the SASRec model
class SASRec(tf.keras.Model):
    def __init__(self, input_dim, sequence_length, embedding_dim, num_heads, num_blocks):
        super(SASRec, self).__init__()
        self.embedding_layer = tf.keras.layers.Embedding(input_dim, embedding_dim, mask_zero=True)
        self.encoder = [
            tf.keras.layers.MultiHeadAttention(
                num_heads=num_heads,
                key_dim=embedding_dim,
                dropout=0.1
            )
            for _ in range(num_blocks)
        ]
        self.dense = tf.keras.layers.Dense(input_dim, activation="softmax")
        self.sequence_length = sequence_length

    def call(self, inputs):
        x = self.embedding_layer(inputs)
        for block in self.encoder:
            x = block(x, x)
        # Focus on the last timestep for prediction
        x = x[:, -1, :]  # Shape becomes (batch_size, embedding_dim)
        return self.dense(x)  # Shape becomes (batch_size, input_dim)

# Build the SASRec model dynamically
model = SASRec(
    input_dim=num_items,  # Updated vocabulary size
    sequence_length=30,
    embedding_dim=50,
    num_heads=2,
    num_blocks=2
)

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=SparseCategoricalCrossentropy(),
    metrics=[SparseTopKCategoricalAccuracy(k=10)]
)

# Training Loop
epochs = 10
batch_size = 32
metrics_history = []

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    start_time = time.time()
    epoch_loss = 0
    steps_done = 0
    total_steps = len(X_train) // batch_size + (len(X_train) % batch_size != 0)

    with tqdm(total=total_steps, desc=f"Epoch {epoch + 1}", dynamic_ncols=True) as pbar:
        for step in range(0, len(X_train), batch_size):
            batch_x = X_train[step:step + batch_size]
            batch_y = y_train[step:step + batch_size]

            # Convert to tensors
            batch_x = tf.convert_to_tensor(batch_x, dtype=tf.int32)
            batch_y = tf.convert_to_tensor(batch_y, dtype=tf.int32)

            try:
                # Train on batch
                metrics = model.train_on_batch(batch_x, batch_y)
                epoch_loss += metrics[0]

                # Update progress dynamically
                pbar.set_postfix(loss=metrics[0])
                pbar.update(1)

            except Exception as e:
                print(f"\nError during training at step {steps_done}: {e}")
                break  # Stop training if an error occurs

    # End of epoch
    elapsed_time = time.time() - start_time
    epoch_avg_loss = epoch_loss / total_steps
    print(f"\nEpoch {epoch + 1} Loss: {epoch_avg_loss:.4f} - Time: {elapsed_time:.2f}s")
    metrics_history.append({"Epoch": epoch + 1, "Loss": epoch_avg_loss, "Time (s)": elapsed_time})

# Model Evaluation
test_loss, test_accuracy = model.evaluate(X_test, y_test, batch_size=batch_size)
print(f"\nTest Loss: {test_loss:.4f}, Test Top-10 Accuracy: {test_accuracy:.4f}")

# Print All Metrics
print("\n=== Final Metrics ===")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Top-10 Accuracy: {test_accuracy:.4f}")
print(f"Training History:")
for metric in metrics_history:
    print(f"Epoch {metric['Epoch']} - Loss: {metric['Loss']:.4f} - Time: {metric['Time (s)']:.2f}s")

# Save Results and Model
results = {
    "Test Loss": test_loss,
    "Test Top-10 Accuracy": test_accuracy
}
results_df = pd.DataFrame([results])
results_df.to_csv(r"C:\Users\user\Desktop\CW4\online+retail\sasrec_results.csv", index=False)
print(f"\nResults saved to sasrec_results.csv")


# Collect all results
all_experiment_results = []

# After each experiment, append results
experiment_details = {
    "Experiment": "Experiment 1",
    "Embedding Dim": 50,
    "Num Heads": 2,
    "Num Blocks": 2,
    "Batch Size": 32,
    "Learning Rate": 0.001,
    "Dropout Rate": 0.1,
    "Epochs": epoch + 1,  # Actual number of epochs run (consider early stopping)
    "Test Loss": test_loss,
    "Test Top-10 Accuracy": test_accuracy
}
all_experiment_results.append(experiment_details)

# At the end of all experiments
results_df = pd.DataFrame(all_experiment_results)
results_df.to_csv(r"C:\Users\user\Desktop\CW4\online+retail\all_experiment_results.csv", index=False)
print("\n=== All Experiment Results ===")
print(results_df)


Epoch 1/10


Epoch 1:   0%|                                                                                 | 0/106 [00:00<?, ?it/s]C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:932: UserWarning: Layer 'query' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:932: UserWarning: Layer 'key' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:932: UserWarning: Layer 'value' (of type EinsumDense) was passed an input with a 

Epoch 1:   5%|██▋                                                       | 5/106 [00:03<00:53,  1.89it/s, loss=8.142521]

Epoch 1: 100%|███████████████████████████████████████████████████████| 106/106 [00:44<00:00,  2.36it/s, loss=7.7747364]



Epoch 1 Loss: 7.9652 - Time: 44.92s
Epoch 2/10


...


Epoch 8 Loss: 6.6444 - Time: 57.26s
Epoch 9/10


Epoch 9: 100%|███████████████████████████████████████████████████████| 106/106 [01:00<00:00,  1.76it/s, loss=6.6106253]



Epoch 9 Loss: 6.6081 - Time: 60.37s
Epoch 10/10


Epoch 10: 100%|██████████████████████████████████████████████████████| 106/106 [01:01<00:00,  1.71it/s, loss=6.5780797]



Epoch 10 Loss: 6.5763 - Time: 61.97s
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 8.6793 - sparse_top_k_categorical_accuracy: 0.0487

Test Loss: 8.6643, Test Top-10 Accuracy: 0.0676

=== Final Metrics ===
Test Loss: 8.6643
Test Top-10 Accuracy: 0.0676
Training History:
Epoch 1 - Loss: 7.9652 - Time: 44.92s
Epoch 2 - Loss: 7.3664 - Time: 45.43s
Epoch 3 - Loss: 7.0499 - Time: 49.40s
Epoch 4 - Loss: 6.8929 - Time: 50.95s
Epoch 5 - Loss: 6.8009 - Time: 51.53s
Epoch 6 - Loss: 6.7375 - Time: 53.70s
Epoch 7 - Loss: 6.6868 - Time: 55.56s
Epoch 8 - Loss: 6.6444 - Time: 57.26s
Epoch 9 - Loss: 6.6081 - Time: 60.37s
Epoch 10 - Loss: 6.5763 - Time: 61.97s

Results saved to sasrec_results.csv

=== All Experiment Results ===
     Experiment  Embedding Dim  Num Heads  Num Blocks  Batch Size  \
0  Experiment 1             50          2           2          32   

   Learning Rate  Dropout Rate  Epochs  Test Loss  Test Top-10 Accuracy  
0          0.001           0.1      10    8.66431              

In [ ]:
# CPU Optimization Settings
import os
os.environ['OMP_NUM_THREADS'] = '16'
os.environ['TF_NUM_INTRA_OP_PARALLELISM_THREADS'] = '16'
os.environ['TF_NUM_INTER_OP_PARALLELISM_THREADS'] = '8'

import time
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseTopKCategoricalAccuracy
from tqdm import tqdm
import pandas as pd

# Define the SASRec model
class SASRec(tf.keras.Model):
    def __init__(self, input_dim, sequence_length, embedding_dim, num_heads, num_blocks):
        super(SASRec, self).__init__()
        self.embedding_layer = tf.keras.layers.Embedding(input_dim, embedding_dim, mask_zero=True)
        self.encoder = [
            tf.keras.layers.MultiHeadAttention(
                num_heads=num_heads,
                key_dim=embedding_dim,
                dropout=0.2
            )
            for _ in range(num_blocks)
        ]
        self.dense = tf.keras.layers.Dense(input_dim, activation="softmax")
        self.sequence_length = sequence_length

    def call(self, inputs):
        x = self.embedding_layer(inputs)
        for block in self.encoder:
            x = block(x, x)
        # Focus on the last timestep for prediction
        x = x[:, -1, :]  # Shape becomes (batch_size, embedding_dim)
        return self.dense(x)  # Shape becomes (batch_size, input_dim)

# Build the SASRec model dynamically
model = SASRec(
    input_dim=num_items,  # Updated vocabulary size
    sequence_length=30,
    embedding_dim=50,
    num_heads=4,
    num_blocks=2
)

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=SparseCategoricalCrossentropy(),
    metrics=[SparseTopKCategoricalAccuracy(k=10)]
)

# Training Loop
epochs = 20
batch_size = 64
metrics_history = []

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    start_time = time.time()
    epoch_loss = 0
    steps_done = 0
    total_steps = len(X_train) // batch_size + (len(X_train) % batch_size != 0)

    with tqdm(total=total_steps, desc=f"Epoch {epoch + 1}", dynamic_ncols=True) as pbar:
        for step in range(0, len(X_train), batch_size):
            batch_x = X_train[step:step + batch_size]
            batch_y = y_train[step:step + batch_size]

            # Convert to tensors
            batch_x = tf.convert_to_tensor(batch_x, dtype=tf.int32)
            batch_y = tf.convert_to_tensor(batch_y, dtype=tf.int32)

            try:
                # Train on batch
                metrics = model.train_on_batch(batch_x, batch_y)
                epoch_loss += metrics[0]

                # Update progress dynamically
                pbar.set_postfix(loss=metrics[0])
                pbar.update(1)

            except Exception as e:
                print(f"\nError during training at step {steps_done}: {e}")
                break  # Stop training if an error occurs

    # End of epoch
    elapsed_time = time.time() - start_time
    epoch_avg_loss = epoch_loss / total_steps
    print(f"\nEpoch {epoch + 1} Loss: {epoch_avg_loss:.4f} - Time: {elapsed_time:.2f}s")
    metrics_history.append({"Epoch": epoch + 1, "Loss": epoch_avg_loss, "Time (s)": elapsed_time})

# Model Evaluation
test_loss, test_accuracy = model.evaluate(X_test, y_test, batch_size=batch_size)
print(f"\nTest Loss: {test_loss:.4f}, Test Top-10 Accuracy: {test_accuracy:.4f}")

# Print All Metrics
print("\n=== Final Metrics ===")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Top-10 Accuracy: {test_accuracy:.4f}")
print(f"Training History:")
for metric in metrics_history:
    print(f"Epoch {metric['Epoch']} - Loss: {metric['Loss']:.4f} - Time: {metric['Time (s)']:.2f}s")

# Save Results and Model
results = {
    "Test Loss": test_loss,
    "Test Top-10 Accuracy": test_accuracy
}
results_df = pd.DataFrame([results])
results_df.to_csv(r"C:\Users\user\Desktop\CW4\online+retail\sasrec_results2.csv", index=False)
print(f"\nResults saved to sasrec_results2.csv")

# Collect all results
all_experiment_results = []

# After each experiment, append results
experiment_details = {
    "Experiment": "Experiment 2",
    "Embedding Dim": 50,
    "Num Heads": 4,
    "Num Blocks": 2,
    "Batch Size": 64,
    "Learning Rate": 0.001,
    "Dropout Rate": 0.2,
    "Epochs": epoch + 1,  # Actual number of epochs run (consider early stopping)
    "Test Loss": test_loss,
    "Test Top-10 Accuracy": test_accuracy
}
all_experiment_results.append(experiment_details)

# At the end of all experiments
results_df = pd.DataFrame(all_experiment_results)
results_df.to_csv(r"C:\Users\user\Desktop\CW4\online+retail\all_experiment_results.csv", index=False)
print("\n=== All Experiment Results ===")
print(results_df)



Epoch 1/20


Epoch 1:   0%|                                                                                  | 0/53 [00:00<?, ?it/s]C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:932: UserWarning: Layer 'query' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:932: UserWarning: Layer 'key' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:932: UserWarning: Layer 'value' (of type EinsumDense) was passed an input with a 


Epoch 1 Loss: 8.0357 - Time: 26.82s
Epoch 2/20


Epoch 2: 100%|██████████████████████████████████████████████████████████| 53/53 [00:25<00:00,  2.08it/s, loss=7.349665]



Epoch 2 Loss: 7.4980 - Time: 25.50s
Epoch 3/20


...


Epoch 18 Loss: 6.3891 - Time: 38.76s
Epoch 19/20


Epoch 19: 100%|█████████████████████████████████████████████████████████| 53/53 [00:38<00:00,  1.37it/s, loss=6.362783]



Epoch 19 Loss: 6.3641 - Time: 38.57s
Epoch 20/20


Epoch 20: 100%|█████████████████████████████████████████████████████████| 53/53 [00:39<00:00,  1.33it/s, loss=6.337656]



Epoch 20 Loss: 6.3407 - Time: 39.80s
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - loss: 10.1364 - sparse_top_k_categorical_accuracy: 0.0391

Test Loss: 10.2381, Test Top-10 Accuracy: 0.0534

=== Final Metrics ===
Test Loss: 10.2381
Test Top-10 Accuracy: 0.0534
Training History:
Epoch 1 - Loss: 8.0357 - Time: 26.82s
Epoch 2 - Loss: 7.4980 - Time: 25.50s
Epoch 3 - Loss: 7.1512 - Time: 53.86s
Epoch 4 - Loss: 6.9727 - Time: 26.48s
Epoch 5 - Loss: 6.8627 - Time: 29.15s
Epoch 6 - Loss: 6.7848 - Time: 30.45s
Epoch 7 - Loss: 6.7244 - Time: 31.72s
Epoch 8 - Loss: 6.6737 - Time: 32.66s
Epoch 9 - Loss: 6.6296 - Time: 34.74s
Epoch 10 - Loss: 6.5920 - Time: 36.59s
Epoch 11 - Loss: 6.5598 - Time: 33.46s
Epoch 12 - Loss: 6.5331 - Time: 33.69s
Epoch 13 - Loss: 6.5095 - Time: 34.91s
Epoch 14 - Loss: 6.4882 - Time: 35.46s
Epoch 15 - Loss: 6.4675 - Time: 36.41s
Epoch 16 - Loss: 6.4430 - Time: 36.90s
Epoch 17 - Loss: 6.4171 - Time: 37.33s
Epoch 18 - Loss: 6.3891 - Time: 38.76s
Epoch 19 - Loss: 6.3641 - Time

In [ ]:
# CPU Optimization Settings
import os
os.environ['OMP_NUM_THREADS'] = '16'
os.environ['TF_NUM_INTRA_OP_PARALLELISM_THREADS'] = '16'
os.environ['TF_NUM_INTER_OP_PARALLELISM_THREADS'] = '8'

import time
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseTopKCategoricalAccuracy
from tqdm import tqdm
import pandas as pd

# Define the SASRec model
class SASRec(tf.keras.Model):
    def __init__(self, input_dim, sequence_length, embedding_dim, num_heads, num_blocks):
        super(SASRec, self).__init__()
        self.embedding_layer = tf.keras.layers.Embedding(input_dim, embedding_dim, mask_zero=True)
        self.encoder = [
            tf.keras.layers.MultiHeadAttention(
                num_heads=num_heads,
                key_dim=embedding_dim,
                dropout=0.1
            )
            for _ in range(num_blocks)
        ]
        self.dense = tf.keras.layers.Dense(input_dim, activation="softmax")
        self.sequence_length = sequence_length

    def call(self, inputs):
        x = self.embedding_layer(inputs)
        for block in self.encoder:
            x = block(x, x)
        # Focus on the last timestep for prediction
        x = x[:, -1, :]  # Shape becomes (batch_size, embedding_dim)
        return self.dense(x)  # Shape becomes (batch_size, input_dim)

# Build the SASRec model dynamically
model = SASRec(
    input_dim=num_items,  # Updated vocabulary size
    sequence_length=30,
    embedding_dim=50,
    num_heads=4,
    num_blocks=2
)

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=SparseCategoricalCrossentropy(),
    metrics=[SparseTopKCategoricalAccuracy(k=10)]
)

# Training Loop
epochs = 15
batch_size = 16
metrics_history = []

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    start_time = time.time()
    epoch_loss = 0
    steps_done = 0
    total_steps = len(X_train) // batch_size + (len(X_train) % batch_size != 0)

    with tqdm(total=total_steps, desc=f"Epoch {epoch + 1}", dynamic_ncols=True) as pbar:
        for step in range(0, len(X_train), batch_size):
            batch_x = X_train[step:step + batch_size]
            batch_y = y_train[step:step + batch_size]

            # Convert to tensors
            batch_x = tf.convert_to_tensor(batch_x, dtype=tf.int32)
            batch_y = tf.convert_to_tensor(batch_y, dtype=tf.int32)

            try:
                # Train on batch
                metrics = model.train_on_batch(batch_x, batch_y)
                epoch_loss += metrics[0]

                # Update progress dynamically
                pbar.set_postfix(loss=metrics[0])
                pbar.update(1)

            except Exception as e:
                print(f"\nError during training at step {steps_done}: {e}")
                break  # Stop training if an error occurs

    # End of epoch
    elapsed_time = time.time() - start_time
    epoch_avg_loss = epoch_loss / total_steps
    print(f"\nEpoch {epoch + 1} Loss: {epoch_avg_loss:.4f} - Time: {elapsed_time:.2f}s")
    metrics_history.append({"Epoch": epoch + 1, "Loss": epoch_avg_loss, "Time (s)": elapsed_time})

# Model Evaluation
test_loss, test_accuracy = model.evaluate(X_test, y_test, batch_size=batch_size)
print(f"\nTest Loss: {test_loss:.4f}, Test Top-10 Accuracy: {test_accuracy:.4f}")

# Print All Metrics
print("\n=== Final Metrics ===")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Top-10 Accuracy: {test_accuracy:.4f}")
print(f"Training History:")
for metric in metrics_history:
    print(f"Epoch {metric['Epoch']} - Loss: {metric['Loss']:.4f} - Time: {metric['Time (s)']:.2f}s")

# Save Results and Model
results = {
    "Test Loss": test_loss,
    "Test Top-10 Accuracy": test_accuracy
}
results_df = pd.DataFrame([results])
results_df.to_csv(r"C:\Users\user\Desktop\CW4\online+retail\sasrec_results5.csv", index=False)
print(f"\nResults saved to sasrec_results5.csv")


# Collect all results
all_experiment_results = []

# After each experiment, append results
experiment_details = {
    "Experiment": "Experiment 5",
    "Embedding Dim": 50,
    "Num Heads": 4,
    "Num Blocks": 2,
    "Batch Size": 16,
    "Learning Rate": 0.001,
    "Dropout Rate": 0.1,
    "Epochs": epoch + 1,  # Actual number of epochs run (consider early stopping)
    "Test Loss": test_loss,
    "Test Top-10 Accuracy": test_accuracy
}
all_experiment_results.append(experiment_details)

# At the end of all experiments
results_df = pd.DataFrame(all_experiment_results)
results_df.to_csv(r"C:\Users\user\Desktop\CW4\online+retail\all_experiment_results.csv", index=False)
print("\n=== All Experiment Results ===")
print(results_df)


Epoch 1/15


Epoch 1:   0%|                                                                                 | 0/211 [00:00<?, ?it/s]C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:932: UserWarning: Layer 'query' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:932: UserWarning: Layer 'key' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:932: UserWarning: Layer 'value' (of type EinsumDense) was passed an input with a 

Epoch 1:   2%|█▎                                                        | 5/211 [00:03<01:43,  2.00it/s, loss=8.142685]

Epoch 1: 100%|███████████████████████████████████████████████████████| 211/211 [01:27<00:00,  2.41it/s, loss=7.7376595]



Epoch 1 Loss: 7.8422 - Time: 87.64s
Epoch 2/15


...


Epoch 13 Loss: 6.1521 - Time: 191.70s
Epoch 14/15


Epoch 14: 100%|███████████████████████████████████████████████████████| 211/211 [03:21<00:00,  1.05it/s, loss=6.073191]



Epoch 14 Loss: 6.0940 - Time: 201.51s
Epoch 15/15


Epoch 15: 100%|██████████████████████████████████████████████████████| 211/211 [03:32<00:00,  1.01s/it, loss=6.0235033]



Epoch 15 Loss: 6.0415 - Time: 212.41s
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 12.0060 - sparse_top_k_categorical_accuracy: 0.0330

Test Loss: 12.4415, Test Top-10 Accuracy: 0.0439

=== Final Metrics ===
Test Loss: 12.4415
Test Top-10 Accuracy: 0.0439
Training History:
Epoch 1 - Loss: 7.8422 - Time: 87.64s
Epoch 2 - Loss: 7.2660 - Time: 93.20s
Epoch 3 - Loss: 6.9847 - Time: 109.42s
Epoch 4 - Loss: 6.8326 - Time: 116.94s
Epoch 5 - Loss: 6.7378 - Time: 117.24s
Epoch 6 - Loss: 6.6509 - Time: 126.85s
Epoch 7 - Loss: 6.5713 - Time: 137.43s
Epoch 8 - Loss: 6.4927 - Time: 144.86s
Epoch 9 - Loss: 6.4105 - Time: 154.99s
Epoch 10 - Loss: 6.3350 - Time: 162.90s
Epoch 11 - Loss: 6.2710 - Time: 172.72s
Epoch 12 - Loss: 6.2115 - Time: 183.13s
Epoch 13 - Loss: 6.1521 - Time: 191.70s
Epoch 14 - Loss: 6.0940 - Time: 201.51s
Epoch 15 - Loss: 6.0415 - Time: 212.41s

Results saved to sasrec_results5.csv

=== All Experiment Results ===
     Experiment  Embedding Dim  Num Heads  Num Blocks  Batch Si

In [ ]:
# CPU Optimization Settings
import os
os.environ['OMP_NUM_THREADS'] = '16'
os.environ['TF_NUM_INTRA_OP_PARALLELISM_THREADS'] = '16'
os.environ['TF_NUM_INTER_OP_PARALLELISM_THREADS'] = '8'

import time
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseTopKCategoricalAccuracy
from tqdm import tqdm
import pandas as pd

# Define the SASRec model
class SASRec(tf.keras.Model):
    def __init__(self, input_dim, sequence_length, embedding_dim, num_heads, num_blocks):
        super(SASRec, self).__init__()
        self.embedding_layer = tf.keras.layers.Embedding(input_dim, embedding_dim, mask_zero=True)
        self.encoder = [
            tf.keras.layers.MultiHeadAttention(
                num_heads=num_heads,
                key_dim=embedding_dim,
                dropout=0.1
            )
            for _ in range(num_blocks)
        ]
        self.dense = tf.keras.layers.Dense(input_dim, activation="softmax")
        self.sequence_length = sequence_length

    def call(self, inputs):
        x = self.embedding_layer(inputs)
        for block in self.encoder:
            x = block(x, x)
        # Focus on the last timestep for prediction
        x = x[:, -1, :]  # Shape becomes (batch_size, embedding_dim)
        return self.dense(x)  # Shape becomes (batch_size, input_dim)

# Build the SASRec model dynamically
model = SASRec(
    input_dim=num_items,  # Updated vocabulary size
    sequence_length=30,
    embedding_dim=50,
    num_heads=4,
    num_blocks=2
)

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=SparseCategoricalCrossentropy(),
    metrics=[SparseTopKCategoricalAccuracy(k=10)]
)

# Early stopping parameters
patience = 5
min_delta = 0.001  # Minimum improvement to reset patience
best_loss = float('inf')
early_stop_counter = 0

# Training Loop
epochs = 50
batch_size = 32
metrics_history = []

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    start_time = time.time()
    epoch_loss = 0
    steps_done = 0
    total_steps = len(X_train) // batch_size + (len(X_train) % batch_size != 0)

    with tqdm(total=total_steps, desc=f"Epoch {epoch + 1}", dynamic_ncols=True) as pbar:
        for step in range(0, len(X_train), batch_size):
            batch_x = X_train[step:step + batch_size]
            batch_y = y_train[step:step + batch_size]

            # Convert to tensors
            batch_x = tf.convert_to_tensor(batch_x, dtype=tf.int32)
            batch_y = tf.convert_to_tensor(batch_y, dtype=tf.int32)

            try:
                # Train on batch
                metrics = model.train_on_batch(batch_x, batch_y)
                epoch_loss += metrics[0]

                # Update progress dynamically
                pbar.set_postfix(loss=metrics[0])
                pbar.update(1)

            except Exception as e:
                print(f"\nError during training at step {steps_done}: {e}")
                break  # Stop training if an error occurs

    # End of epoch
    elapsed_time = time.time() - start_time
    epoch_avg_loss = epoch_loss / total_steps
    print(f"\nEpoch {epoch + 1} Loss: {epoch_avg_loss:.4f} - Time: {elapsed_time:.2f}s")
    metrics_history.append({"Epoch": epoch + 1, "Loss": epoch_avg_loss, "Time (s)": elapsed_time})

    # Early stopping logic
    if epoch_avg_loss < best_loss - min_delta:
        best_loss = epoch_avg_loss
        early_stop_counter = 0  # Reset patience counter
    else:
        early_stop_counter += 1

    if early_stop_counter >= patience:
        print(f"\nEarly stopping triggered. Stopping training at epoch {epoch + 1}.")
        break

# Model Evaluation
test_loss, test_accuracy = model.evaluate(X_test, y_test, batch_size=batch_size)
print(f"\nTest Loss: {test_loss:.4f}, Test Top-10 Accuracy: {test_accuracy:.4f}")

# Print All Metrics
print("\n=== Final Metrics ===")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Top-10 Accuracy: {test_accuracy:.4f}")
print(f"Training History:")
for metric in metrics_history:
    print(f"Epoch {metric['Epoch']} - Loss: {metric['Loss']:.4f} - Time: {metric['Time (s)']:.2f}s")

# Save Results and Model
results = {
    "Test Loss": test_loss,
    "Test Top-10 Accuracy": test_accuracy
}
results_df = pd.DataFrame([results])
results_df.to_csv(r"C:\Users\user\Desktop\CW4\online+retail\sasrec_results6.csv", index=False)
print(f"\nResults saved to sasrec_results6.csv")


# Collect all results
all_experiment_results = []

# After each experiment, append results
experiment_details = {
    "Experiment": "Experiment 6",
    "Embedding Dim": 50,
    "Num Heads": 4,
    "Num Blocks": 2,
    "Batch Size": 32,
    "Learning Rate": 0.001,
    "Dropout Rate": 0.1,
    "Epochs": epoch + 1,  # Actual number of epochs run (consider early stopping)
    "Test Loss": test_loss,
    "Test Top-10 Accuracy": test_accuracy
}
all_experiment_results.append(experiment_details)

# At the end of all experiments
results_df = pd.DataFrame(all_experiment_results)
results_df.to_csv(r"C:\Users\user\Desktop\CW4\online+retail\all_experiment_results.csv", index=False)
print("\n=== All Experiment Results ===")
print(results_df)


Epoch 1/50


Epoch 1:   0%|                                                                                 | 0/106 [00:00<?, ?it/s]C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:932: UserWarning: Layer 'query' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:932: UserWarning: Layer 'key' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:932: UserWarning: Layer 'value' (of type EinsumDense) was passed an input with a 


Epoch 1 Loss: 7.9483 - Time: 59.44s
Epoch 2/50


Epoch 2: 100%|█████████████████████████████████████████████████████████| 106/106 [01:00<00:00,  1.76it/s, loss=7.26222]



Epoch 2 Loss: 7.3535 - Time: 60.15s
Epoch 3/50


...


Epoch 48 Loss: 5.4667 - Time: 224.90s
Epoch 49/50


Epoch 49: 100%|██████████████████████████████████████████████████████| 106/106 [03:48<00:00,  2.15s/it, loss=5.3974414]



Epoch 49 Loss: 5.4199 - Time: 228.18s
Epoch 50/50


Epoch 50: 100%|██████████████████████████████████████████████████████| 106/106 [03:51<00:00,  2.18s/it, loss=5.3473253]



Epoch 50 Loss: 5.3711 - Time: 231.11s
27/27 ━━━━━━━━━━━━━━━━━━━━ 7s 238ms/step - loss: 25.7803 - sparse_top_k_categorical_accuracy: 0.0618

Test Loss: 26.4220, Test Top-10 Accuracy: 0.0569

=== Final Metrics ===
Test Loss: 26.4220
Test Top-10 Accuracy: 0.0569
Training History:
Epoch 1 - Loss: 7.9483 - Time: 59.44s
Epoch 2 - Loss: 7.3535 - Time: 60.15s
Epoch 3 - Loss: 7.0383 - Time: 61.27s
Epoch 4 - Loss: 6.8891 - Time: 63.82s
Epoch 5 - Loss: 6.8177 - Time: 65.67s
Epoch 6 - Loss: 6.7713 - Time: 69.04s
Epoch 7 - Loss: 6.7297 - Time: 71.47s
Epoch 8 - Loss: 6.6933 - Time: 453.40s
Epoch 9 - Loss: 6.6577 - Time: 70.13s
Epoch 10 - Loss: 6.6232 - Time: 70.02s
Epoch 11 - Loss: 6.5872 - Time: 79.47s
Epoch 12 - Loss: 6.5488 - Time: 92.80s
Epoch 13 - Loss: 6.5100 - Time: 106.92s
Epoch 14 - Loss: 6.4738 - Time: 120.01s
Epoch 15 - Loss: 6.4394 - Time: 134.08s
Epoch 16 - Loss: 6.4048 - Time: 147.63s
Epoch 17 - Loss: 6.3728 - Time: 157.32s
Epoch 18 - Loss: 6.3409 - Time: 165.26s
Epoch 19 - Loss: 6.31

In [ ]:
# CPU Optimization Settings
import os
os.environ['OMP_NUM_THREADS'] = '16'
os.environ['TF_NUM_INTRA_OP_PARALLELISM_THREADS'] = '16'
os.environ['TF_NUM_INTER_OP_PARALLELISM_THREADS'] = '8'

import time
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseTopKCategoricalAccuracy
from tqdm import tqdm
import pandas as pd

# Define the SASRec model
class SASRec(tf.keras.Model):
    def __init__(self, input_dim, sequence_length, embedding_dim, num_heads, num_blocks):
        super(SASRec, self).__init__()
        self.embedding_layer = tf.keras.layers.Embedding(input_dim, embedding_dim, mask_zero=True)
        self.encoder = [
            tf.keras.layers.MultiHeadAttention(
                num_heads=num_heads,
                key_dim=embedding_dim,
                dropout=0.3
            )
            for _ in range(num_blocks)
        ]
        self.dense = tf.keras.layers.Dense(input_dim, activation="softmax")
        self.sequence_length = sequence_length

    def call(self, inputs):
        x = self.embedding_layer(inputs)
        for block in self.encoder:
            x = block(x, x)
        # Focus on the last timestep for prediction
        x = x[:, -1, :]  # Shape becomes (batch_size, embedding_dim)
        return self.dense(x)  # Shape becomes (batch_size, input_dim)

# Build the SASRec model dynamically
model = SASRec(
    input_dim=num_items,  # Updated vocabulary size
    sequence_length=30,
    embedding_dim=200,
    num_heads=4,
    num_blocks=2
)

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=SparseCategoricalCrossentropy(),
    metrics=[SparseTopKCategoricalAccuracy(k=10)]
)

# Training Loop
epochs = 30
batch_size = 64
metrics_history = []

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    start_time = time.time()
    epoch_loss = 0
    steps_done = 0
    total_steps = len(X_train) // batch_size + (len(X_train) % batch_size != 0)

    with tqdm(total=total_steps, desc=f"Epoch {epoch + 1}", dynamic_ncols=True) as pbar:
        for step in range(0, len(X_train), batch_size):
            batch_x = X_train[step:step + batch_size]
            batch_y = y_train[step:step + batch_size]

            # Convert to tensors
            batch_x = tf.convert_to_tensor(batch_x, dtype=tf.int32)
            batch_y = tf.convert_to_tensor(batch_y, dtype=tf.int32)

            try:
                # Train on batch
                metrics = model.train_on_batch(batch_x, batch_y)
                epoch_loss += metrics[0]

                # Update progress dynamically
                pbar.set_postfix(loss=metrics[0])
                pbar.update(1)

            except Exception as e:
                print(f"\nError during training at step {steps_done}: {e}")
                break  # Stop training if an error occurs

    # End of epoch
    elapsed_time = time.time() - start_time
    epoch_avg_loss = epoch_loss / total_steps
    print(f"\nEpoch {epoch + 1} Loss: {epoch_avg_loss:.4f} - Time: {elapsed_time:.2f}s")
    metrics_history.append({"Epoch": epoch + 1, "Loss": epoch_avg_loss, "Time (s)": elapsed_time})

# Model Evaluation
test_loss, test_accuracy = model.evaluate(X_test, y_test, batch_size=batch_size)
print(f"\nTest Loss: {test_loss:.4f}, Test Top-10 Accuracy: {test_accuracy:.4f}")

# Print All Metrics
print("\n=== Final Metrics ===")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Top-10 Accuracy: {test_accuracy:.4f}")
print(f"Training History:")
for metric in metrics_history:
    print(f"Epoch {metric['Epoch']} - Loss: {metric['Loss']:.4f} - Time: {metric['Time (s)']:.2f}s")

# Save Results and Model
results = {
    "Test Loss": test_loss,
    "Test Top-10 Accuracy": test_accuracy
}
results_df = pd.DataFrame([results])
results_df.to_csv(r"C:\Users\user\Desktop\CW4\online+retail\sasrec_results8.csv", index=False)
print(f"\nResults saved to sasrec_results8.csv")


# Collect all results
all_experiment_results = []

# After each experiment, append results
experiment_details = {
    "Experiment": "Experiment 1",
    "Embedding Dim": 200,
    "Num Heads": 4,
    "Num Blocks": 2,
    "Batch Size": 64,
    "Learning Rate": 0.001,
    "Dropout Rate": 0.3,
    "Epochs": epoch + 1,  # Actual number of epochs run (consider early stopping)
    "Test Loss": test_loss,
    "Test Top-10 Accuracy": test_accuracy
}
all_experiment_results.append(experiment_details)

# At the end of all experiments
results_df = pd.DataFrame(all_experiment_results)
results_df.to_csv(r"C:\Users\user\Desktop\CW4\online+retail\all_experiment_results.csv", index=False)
print("\n=== All Experiment Results ===")
print(results_df)


Epoch 1/30


Epoch 1:   0%|                                                                                  | 0/53 [00:00<?, ?it/s]C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:932: UserWarning: Layer 'query' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:932: UserWarning: Layer 'key' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:932: UserWarning: Layer 'value' (of type EinsumDense) was passed an input with a 

Epoch 1:   9%|█████▌                                                     | 5/53 [00:04<00:28,  1.66it/s, loss=8.102575]

Epoch 1: 100%|█████████████████████████████████████████████████████████| 53/53 [00:27<00:00,  1.91it/s, loss=7.7736883]



Epoch 1 Loss: 7.9462 - Time: 27.73s
Epoch 2/30


...


Epoch 28 Loss: 6.1057 - Time: 69.48s
Epoch 29/30


Epoch 29: 100%|████████████████████████████████████████████████████████| 53/53 [01:11<00:00,  1.36s/it, loss=6.0463448]



Epoch 29 Loss: 6.0621 - Time: 71.94s
Epoch 30/30


Epoch 30: 100%|████████████████████████████████████████████████████████| 53/53 [01:15<00:00,  1.43s/it, loss=6.0030475]



Epoch 30 Loss: 6.0188 - Time: 75.89s
14/14 ━━━━━━━━━━━━━━━━━━━━ 8s 566ms/step - loss: 13.2116 - sparse_top_k_categorical_accuracy: 0.0327

Test Loss: 13.3522, Test Top-10 Accuracy: 0.0356

=== Final Metrics ===
Test Loss: 13.3522
Test Top-10 Accuracy: 0.0356
Training History:
Epoch 1 - Loss: 7.9462 - Time: 27.73s
Epoch 2 - Loss: 7.4111 - Time: 26.70s
Epoch 3 - Loss: 7.0929 - Time: 27.20s
Epoch 4 - Loss: 6.9824 - Time: 27.71s
Epoch 5 - Loss: 6.9281 - Time: 28.27s
Epoch 6 - Loss: 6.8672 - Time: 29.90s
Epoch 7 - Loss: 6.8212 - Time: 30.62s
Epoch 8 - Loss: 6.7788 - Time: 31.82s
Epoch 9 - Loss: 6.7400 - Time: 32.21s
Epoch 10 - Loss: 6.7010 - Time: 35.01s
Epoch 11 - Loss: 6.6639 - Time: 36.41s
Epoch 12 - Loss: 6.6378 - Time: 37.55s
Epoch 13 - Loss: 6.6146 - Time: 39.28s
Epoch 14 - Loss: 6.5842 - Time: 41.11s
Epoch 15 - Loss: 6.5561 - Time: 42.56s
Epoch 16 - Loss: 6.5227 - Time: 45.10s
Epoch 17 - Loss: 6.4972 - Time: 46.96s
Epoch 18 - Loss: 6.4747 - Time: 48.10s
Epoch 19 - Loss: 6.4569 - Tim